In [ ]:
# imports
import re
import time
import pickle
import logging
import gc
import os
import math
import functools

import pandas as pd
import numpy as np
import math as m
import matplotlib.pyplot as plt

import tensorflow as tf

from scipy import stats

from six.moves import xrange 


log = logging.getLogger('log')
log.setLevel(logging.DEBUG)

lhnd = logging.StreamHandler()
lhnd.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
lhnd.setFormatter(formatter)

log.addHandler(lhnd)

In [ ]:
def lmap(f, arr):
    return list(map(f, arr))

def lfilter(f, arr):
    return list(filter(f, arr))

def foreach(it, f):
    for e in it:
        f(e)

In [ ]:
total = 947528

def raw_data_filter(file):
    # Mapping to events
    res = list()

    i = 0
    
    for line in file:
        cur = line.rstrip().split(',')
        cur = lmap(lambda p: (re.sub(';.*', '', p), re.sub('.*;', '', p)), cur)

        session = list()
        
        for j in range(0, len(cur)):
            try:
                session.append(int(cur[j][1]))
            except ValueError:
                None
                
        res.append(session)

        i = i + 1
                
        if (i % 100000 == 0):
            gc.collect()

            log.debug("%d %% of mapping is done.", i / total * 100)

    
    return res

In [ ]:
raw_data = raw_data_filter(open("data/sessions_public.txt","r"))
    


In [ ]:
def group_count(data):
    total = dict()

    for i in data:
        for j in i:
            if (j in total.keys()):
                total[j] = total[j] + 1
            else:
                total[j] = 1
                
    return total

In [ ]:
def set_map(lst, cnt, min_allowed, min_session_size):
    result = []
    groups = set()
    
    for session in lst:
        unsub = set()
        sub = set()
        
        for event in session:
            if (event < 0):
                sub_event = -event
                
                if (sub_event in sub):
                    sub.remove(sub_event)
                    
                unsub.add(sub_event)
            else:
                if (not event in unsub):
                    if (cnt == None or cnt[event] > min_allowed):
                        sub.add(event)
        
        if (len(sub) >= min_session_size):
            for event in sub:
                groups.add(event)
            
            result.append(sub)
    
    return result, groups

def drop_uncommon(raw_data, min_allowed = 10, min_session_size = 4):
    cnt = None
    sorted_cnt = None
    
    data = raw_data
    groups = None
    
    while (cnt == None or sorted_cnt[0] < min_allowed):
        data, groups = set_map(data, cnt, min_allowed, min_session_size)
        
        cnt = group_count(data) 
        sorted_cnt = sorted(list(cnt.values()))
        
        log.info("Length of data:   %d", len(data))
        log.info("Total length:     %d", 
                functools.reduce((lambda x, y: x + y), lmap(lambda a: len(a), data))
                )
        log.info("Number of groups: %d", len(groups))
        log.info("Minimum count:    %d", sorted_cnt[0])
        
    return data, groups

In [ ]:
#data = drop_uncommon(data)
data, groups = drop_uncommon(raw_data, 10)

w2i = {w: i for i, w in enumerate(groups)}
i2w = {i: w for i, w in enumerate(groups)}

In [ ]:
session_dex = 0
event_dex = 0

def generate_batch(batch_size):
    global session_dex
    global event_dex
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    current = 0
    session = list(data[session_dex])
    
    while (current * 2 < batch_size):
        batch[current * 2] = w2i[session[event_dex + 1]]
        labels[current * 2][0] = w2i[session[event_dex]]
        batch[current * 2 + 1] = w2i[session[event_dex + 1]]
        labels[current * 2 + 1][0] = w2i[session[(event_dex + 2) % len(session)]]
        
        event_dex += 1
        current += 1
        
        if (event_dex + 2 >= len(session)):
            event_dex = 0
            session_dex = session_dex + 1
            if (session_dex >= len(data)):
                session_dex = 0
            session = list(data[session_dex])

     
    return batch, labels


In [ ]:
batch, labels = generate_batch(10)

print(data[0])
print(data[1])

for i in range(10):
    print(batch[i], i2w[batch[i]], '->', labels[i, 0],
          i2w[labels[i, 0]])

In [ ]:
batch_size = 128
embedding_size = 128
num_sampled = 64
vocabulary_size = len(groups)

graph = tf.Graph()

In [ ]:
with graph.as_default():
    # Input data.
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(
                tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)
            )
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)

      # Construct the variables for the NCE loss
    with tf.name_scope('weights'):
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
            stddev=1.0 / math.sqrt(embedding_size)))
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    # Explanation of the meaning of NCE loss:
    #   http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                  weights=nce_weights,
                  biases=nce_biases,
                  labels=train_labels,
                  inputs=embed,
                  num_sampled=num_sampled,
                  num_classes=vocabulary_size))
        
        # Add the loss value as a scalar to summary.
    tf.summary.scalar('loss', loss)

    # Construct the SGD optimizer using a learning rate of 1.0.
    with tf.name_scope('optimizer'):
          optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all
    # embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,
    #                                          valid_dataset)
    #similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Merge all summaries.
    merged = tf.summary.merge_all()

    # Add variable initializer.
    init = tf.global_variables_initializer()

    # Create a saver.
    saver = tf.train.Saver()



In [ ]:
num_steps = 133796

with tf.Session(graph=graph) as session:
    # Open a writer to write summaries.
    writer = tf.summary.FileWriter("tmp", session.graph)

    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # Define metadata variable.
        run_metadata = tf.RunMetadata()

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        # Also, evaluate the merged op to get all summaries from the returned
        # "summary" variable. Feed metadata variable to session for visualizing
        # the graph in TensorBoard.
        _, summary, loss_val = session.run([optimizer, merged, loss],
                                         feed_dict=feed_dict,
                                         run_metadata=run_metadata)
        average_loss += loss_val

        # Add returned summaries to writer in each step.
        writer.add_summary(summary, step)
        # Add metadata to visualize the graph for the last run.
        if step == (num_steps - 1):
            writer.add_run_metadata(run_metadata, 'step%d' % step)

        if step % 2000 == 0:
            if step > 0:
                  average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000
            # batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
            
    final_embeddings = normalized_embeddings.eval()



In [21]:
with tf.Session(graph=graph) as session:
    p = final_embeddings[0]
    cnst = tf.constant(p, shape=[1, 128])
    res = tf.matmul(cnst, final_embeddings, transpose_b=True).eval()[0]

    dxs = np.argsort(np.array(res))
    #print(dxs[0],' ',res[dxs[0]])
    for i in range(len(dxs) - 10, len(dxs)):
        print(i2w[dxs[i]],' ',res[dxs[i]])


135768481   0.31091785
19750497   0.3112375
105775670   0.31286514
114945987   0.3211316
45817024   0.3226113
153418524   0.3265404
76276836   0.32681698
46868895   0.36412835
150539414   0.36771876
163053568   1.0
